In [4]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

print('done')

done


In [5]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [6]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [7]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [8]:
import json
with open('natural_question.json') as f: 
    data = json.load(f)
len(data)


5249

In [18]:
count = 0
question_output = {}
for i in range(len(data)): 
    if data[str(i+1)][0]['question_template_id'] == '2q1':
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if 'recovery rate' in ent[0] or 'Case-Fatality rate' in ent[0] or 'testing rate' in ent[0] or 'Incidence Rate' in ent[0]: 
            continue
        print(count)
        if ent[0] == 'percent positive rate' or ent[0] == 'percent postive rate': 
            df = pd.read_csv("https://api.covidtracking.com/v1/states/daily.csv")
            df = df.query("date == '20210307'")
            df['postive rate'] = df['positive'] * 100.0/df['totalTestResults']
            df = df[['state', 'postive rate']]
            df = df.astype(str)
        elif ent[0] == 'percent negative rate': 
            df = pd.read_csv("https://api.covidtracking.com/v1/states/daily.csv")
            df = df.query("date == '20210307'")
            df['negative rate'] = df['negative'] * 100.0/df['totalTestResults']
            df = df[['state', 'negative rate']]
            df = df.astype(str)
        elif ent[0] == 'daily percent positive rate': 
            df = pd.read_csv("https://api.covidtracking.com/v1/states/daily.csv")
            df = df.query("date == '20210307'")
            df['daily percent positive rate'] = df['positiveIncrease'] * 100.0/df['totalTestResultsIncrease']
            df = df[['state', 'daily percent positive rate']]
            df = df.astype(str)
        else: 
            df = pd.read_csv("https://api.covidtracking.com/v1/states/daily.csv")
            df = df.query("date == '20210307'")
            df['hospitalization rate'] = df['hospitalizedCumulative'] * 100.0/df['positive']
            df = df[['state', 'hospitalization rate']]
            df = df.astype(str)
        list_of_list = [[]]
        list_of_list[0] = list(df.columns)
        list_of_list.extend(df.values.tolist())
        list_of_list
        result = predict(list_of_list, [q])
        res = ""
        count1 = 0
        for i in result[0]: 
            if count1 == 0:
                count1 +=1 
                res += list_of_list[i[0]+1][i[1]]
            else: 
                res += ', ' + list_of_list[i[0]+1][i[1]]
            question_output[q] = res
        count += 1
question_output

0
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


state,postive rate
AK,3.2851166647801953
AL,21.50880372908372
AR,11.870085315164728
AS,0.0
AZ,10.450721127248563
CA,7.052719277724894
CO,6.805824299861436
CT,4.3759813482862775
DC,3.283670125094838
DE,6.1702219782644825



> What is the percent postive rate in Ohio?
26.776219423446747
1
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


state,negative rate
AK,nan
AL,83.12767773996595
AR,90.65479918814285
AS,100.0
AZ,38.858993399809435
CA,nan
CO,34.28551564794627
CT,nan
DC,nan
DE,38.065089228474335



> What is the percent negative rate in Ohio?
100.0, 38.065089228474335
2
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


...

state,daily percent positive rate
AK,nan
AL,17.383894333191307
AR,4.881656804733728
AS,nan
AZ,2.9594324983373976
CA,2.8651660084393256
CO,2.2010848203757565
CT,nan
DC,2.549772965420887
DE,3.6645645133799216



> What is the daily percent positive rate in California?
2.8651660084393256
215
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


state,hospitalization rate
AK,2.2729669866047884
AL,9.198529867812148
AR,4.595188690281942
AS,nan
AZ,7.006681557594252
CA,nan
CO,5.4750092761828855
CT,nan
DC,nan
DE,nan



> What is the hospitilization rate in California?
nan


{'What is the percent postive rate in Ohio?': '26.776219423446747',
 'What is the percent negative rate in Ohio?': '100.0, 38.065089228474335',
 'What is the daily percent positive rate in Ohio?': '37300.0',
 'What is the hospitilization rate in Ohio?': '4.737417490242483, 3.4283967331964886',
 'What is the percent postive rate in New Mexico?': '26.776219423446747',
 'What is the percent negative rate in New Mexico?': '100.0',
 'What is the daily percent positive rate in New Mexico?': '37300.0',
 'What is the hospitilization rate in New Mexico?': '3.4283967331964886',
 'What is the percent postive rate in Wisconsin?': '26.776219423446747',
 'What is the percent negative rate in Wisconsin?': '100.0, 38.065089228474335',
 'What is the daily percent positive rate in Wisconsin?': '1.568420125121156, 100.0',
 'What is the hospitilization rate in Wisconsin?': '4.737417490242483',
 'What is the percent postive rate in South Dakota?': '26.776219423446747',
 'What is the percent negative rate i

In [21]:
question_output
print(count)

216


In [20]:
with open('2q1data.json', 'w') as outfile:
    json.dump(question_output, outfile)